In [4]:
from pathlib import Path
import cv2
import numpy as np
import pandas as pd

# ===== CẤU HÌNH =====
DATA_ROOT = Path("Data")

# Manifest Sorensen đã có sẵn
SOR_OUT       = DATA_ROOT / "processed" / "sorensen"
SOR_MANIFEST  = SOR_OUT / "manifest_slices_preprocessed_clean.csv"

# Thư mục chứa các file CLE1.jpg, PLE3.png, PSE10.jpeg...
# → chỉnh lại cho đúng đường dẫn trên máy bạn
MANUAL_RAW_DIR = DATA_ROOT / "raw" / "gathered_raw_data"

# Nơi lưu .npz & manifest mới cho ảnh manual
MANUAL_OUT_DIR = DATA_ROOT / "processed" / "manual_cle_ple_pse"
MANUAL_PRE_DIR = MANUAL_OUT_DIR / "preprocessed"
MANUAL_OUT_DIR.mkdir(parents=True, exist_ok=True)
MANUAL_PRE_DIR.mkdir(parents=True, exist_ok=True)

# Giữ đúng kích thước dùng cho Sorensen
TARGET_SIZE = (512, 512)      # (width, height) – đổi nếu bạn dùng size khác

# Giữ đúng HU range để img_win tương thích
HU_MIN, HU_MAX = -1000, 200

# Map tên nhãn → mã code giống Sorensen
# 1: NT, 2: CLE, 3: PSE, 4: PLE
label_name2code = {"NT": 1, "CLE": 2, "PSE": 3, "PLE": 4}


In [5]:
def preprocess_manual_image(img_path: Path,
                            out_dir: Path,
                            slice_key: str):
    """
    Tiền xử lý 1 ảnh manual (JPG/PNG) thành file .npz
    có cùng cấu trúc với Sorensen:
      - img_norm [H,W] float32 trong [0,1]
      - img_win  [H,W] float32 trong [HU_MIN, HU_MAX]
      - lung_mask [H,W] bool
      - meta: dict (source, orig_path, slice_key)
    """
    # Đọc grayscale
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(img_path)

    # Resize về TARGET_SIZE
    img_resized = cv2.resize(img, TARGET_SIZE,
                             interpolation=cv2.INTER_AREA).astype(np.float32)

    # Chuẩn hóa 0–255 → 0–1
    img_norm = img_resized / 255.0

    # Giả lập HU window: [0,1] → [HU_MIN, HU_MAX]
    img_win = img_norm * (HU_MAX - HU_MIN) + HU_MIN

    # Ở đây đơn giản: mask = 1 hết
    lung_mask = np.ones_like(img_norm, dtype=bool)

    meta = {
        "source": "manual_cle_ple_pse",
        "orig_path": str(img_path),
        "slice_key": slice_key,
    }

    out_path = out_dir / f"{slice_key}.npz"
    np.savez_compressed(
        out_path,
        img_norm=img_norm.astype(np.float32),
        img_win=img_win.astype(np.float32),
        lung_mask=lung_mask,
        meta=meta,
    )
    return out_path


In [6]:
# Quét toàn bộ file ảnh manual và tạo manifest
files = sorted([
    p for p in MANUAL_RAW_DIR.iterdir()
    if p.suffix.lower() in [".jpg", ".jpeg", ".png"]
])

rows = []
BASE_SUBJECT_ID = 10000  # subject_id ảo cho ảnh manual

for idx, img_path in enumerate(files):
    stem = img_path.stem  # ví dụ "CLE1"
    # Tách phần chữ & số
    prefix = "".join([c for c in stem if c.isalpha()]).upper()   # CLE / PLE / PSE
    number = "".join([c for c in stem if c.isdigit()])           # "1"

    if prefix not in ["CLE", "PLE", "PSE"]:
        print(f"Skip {img_path.name} vì prefix lạ:", prefix)
        continue

    num = int(number) if number else (idx + 1)

    # Tạo slice_key + subject_id + label
    slice_key   = f"manual_{prefix}{num:03d}"
    subject_id  = BASE_SUBJECT_ID + idx
    label_name  = prefix
    label_code  = label_name2code[label_name]

    # Preprocess → .npz
    pre_path = preprocess_manual_image(img_path, MANUAL_PRE_DIR, slice_key)

    rows.append({
        "slice_key": slice_key,
        "subject_id": subject_id,
        "label_name": label_name,
        "label_code": label_code,
        "preprocessed_path": pre_path.as_posix(),
    })

df_manual = pd.DataFrame(rows)
manual_manifest_path = MANUAL_OUT_DIR / "manifest_manual_preprocessed_clean.csv"
df_manual.to_csv(manual_manifest_path, index=False)

print("Số mẫu manual:", len(df_manual))
print("Lưu manifest manual tại:", manual_manifest_path)
display(df_manual.head())
print("\nPhân bố lớp manual:")
print(df_manual["label_name"].value_counts())


Số mẫu manual: 46
Lưu manifest manual tại: Data\processed\manual_cle_ple_pse\manifest_manual_preprocessed_clean.csv


,slice_key,subject_id,label_name,label_code,preprocessed_path
0,manual_CLE001,10000,CLE,2,Data/processed/manual_cle_ple_pse/preprocessed...
1,manual_CLE010,10001,CLE,2,Data/processed/manual_cle_ple_pse/preprocessed...
2,manual_CLE011,10002,CLE,2,Data/processed/manual_cle_ple_pse/preprocessed...
3,manual_CLE012,10003,CLE,2,Data/processed/manual_cle_ple_pse/preprocessed...
4,manual_CLE013,10004,CLE,2,Data/processed/manual_cle_ple_pse/preprocessed...



Phân bố lớp manual:
label_name
CLE    22
PLE    14
PSE    10
Name: count, dtype: int64


In [7]:
# Đọc manifest Sorensen gốc
df_sor = pd.read_csv(SOR_MANIFEST)

# Thêm cột nguồn dữ liệu để sau này dễ filter
df_sor["source"]    = "sorensen"
df_manual["source"] = "manual"

# Merge
df_all = pd.concat([df_sor, df_manual], ignore_index=True)

# Lưu ra manifest tổng
OUT_ALL = DATA_ROOT / "processed" / "emphysema_all"
OUT_ALL.mkdir(parents=True, exist_ok=True)
MANIFEST_ALL = OUT_ALL / "manifest_emphysema_all.csv"
df_all.to_csv(MANIFEST_ALL, index=False)

print("Đã lưu manifest tổng tại:", MANIFEST_ALL)
print("\nPhân bố lớp sau khi merge:")
print(df_all["label_name"].value_counts())


Đã lưu manifest tổng tại: Data\processed\emphysema_all\manifest_emphysema_all.csv

Phân bố lớp sau khi merge:
label_name
NT     61
CLE    43
PSE    37
PLE    20
Name: count, dtype: int64


In [8]:
df = pd.read_csv("Data/processed/emphysema_all/manifest_emphysema_all.csv")